In [3]:
import tensorflow as tf
import numpy as np

In [50]:
def dictionaryGen(D):
    
    """ Generate and return 
        - D integers on the x axis evenly distributed around 0 (??? w/ abs value \leq 100 ???)
        - The step size Δx """

    d_pos = np.linspace(0, 100, num= int(D/2)+1, endpoint=False, retstep=True, dtype=np.float32)
    d_neg = (-1*d_pos[0][1:]) # get the negative dictionary points  
    return (d_pos[1], np.concatenate((np.sort(d_neg), d_pos[0][1:]), axis=None))


def gaussianKer1D(s, d_i, b_width):

    """ Compute 1D Gaussian kernel as defined in arXiv:1707.04035 """
    
    x = np.power((s - d_i), 2)
    return np.exp(-b_width*x)

In [51]:
print(tf.reduce_sum(gaussianKer1D(np.array([13.53]), np.array([20, -20]), 0.32)))
gaussianKer1D(np.array([13.53]), np.array([20, -20]), 0.32)

tf.Tensor(1.5219958883602659e-06, shape=(), dtype=float64)


array([1.52199589e-006, 5.71087128e-157])

In [52]:
step, d = dictionaryGen(50)
tf.constant(d)

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([-96.15385  , -92.30769  , -88.46154  , -84.61539  , -80.76923  ,
       -76.92308  , -73.07692  , -69.23077  , -65.38461  , -61.53846  ,
       -57.692307 , -53.846153 , -50.       , -46.153847 , -42.307693 ,
       -38.46154  , -34.615383 , -30.76923  , -26.923077 , -23.076923 ,
       -19.23077  , -15.384615 , -11.538462 ,  -7.6923075,  -3.8461537,
         3.8461537,   7.6923075,  11.538462 ,  15.384615 ,  19.23077  ,
        23.076923 ,  26.923077 ,  30.76923  ,  34.615383 ,  38.46154  ,
        42.307693 ,  46.153847 ,  50.       ,  53.846153 ,  57.692307 ,
        61.53846  ,  65.38461  ,  69.23077  ,  73.07692  ,  76.92308  ,
        80.76923  ,  84.61539  ,  88.46154  ,  92.30769  ,  96.15385  ],
      dtype=float32)>

In [114]:
from tensorflow.keras.layers import Layer

class Kaf(Layer):

    def __init__(self, D):

        super(Kaf, self).__init__()
        
        # Initialize constants
        self.D = D
        step, dict = dictionaryGen(D)
        self.d = tf.constant(dict)
        self.k_bandw = 1/(6*(np.power(step,2)))
        

    def build(self, input_shape):
        
        # Mixing coefficients sampled from a normal dist
        self.a = self.add_weight(shape=(input_shape[-1], self.D), 
                                 initializer= 'random_normal',
                                 trainable=True)

    def call(self, inputs):
        
        y = []
        for i in range (0, inputs.shape[0]): # for each input unit
            y_i = []
            for j in range (0, inputs.shape[-1]): # apply the activation function element-wise
                y_i.append(tf.reduce_sum(self.a[j] * gaussianKer1D(inputs[i][j], self.d, self.k_bandw)))
            y.append(y_i)                
        
        y = tf.stack(y) # convert back to a tensor
        print(y)
        print(x)
        return y 

In [115]:
x = tf.random.uniform(shape = (4,), minval=-20, maxval=40, dtype=tf.float32)

kaf = Kaf(20)

kaf(x)

InvalidArgumentError: Index out of range using input dim 0; input has only 0 dims [Op:StridedSlice] name: kaf_51/strided_slice/